# DMML 25.11.2024: Feedback Übung2 - Gewichtsklasse

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split

## Alle: Feedback

<ul>
<li><b>Gitternetz</b>
    <ul>
    <li>Nur wenige von euch haben Diagramme mit Gitternetz gemacht: plt.grid()
    <li>Aus meiner Sicht erleichert das i.d.R. das Lesen von Diagrammen</li>
    </ul>
<li><b>Positive Klasse (1.Zeile/1.Spalte Konfusionsmatrix)</b>
    <ul>
    <li>Die positive Klasse ist die Klasse, die vom größeren Interesse ist
    <li>Diese sollte mit 0 kodiert werden (war leider in der Angabe anders!)
    <li>Häufig ist die positive Klasse die Klasse zu der es weniger Datensätze gibt
    </ul>
<li><b>Konfidenzmatrix</b>
    <ul>
    <li>Diese sollte immer als Matrix dargestellt sein
    <li>Sollte eine Methode keine Matrix liefern, ist ein anderer Weg zu suchen
    </ul>
<li><b>Präsentationen</b>
<ul>
<li>Zunächst die Aufgabenstellung erklären, dann<br>
    (https://www.inc.com/john-baldoni/deliver-a-great-speech-aristotle-three-tips.html)
    <ol>
    <li>Tell them what you will tell them
    <li>Tell them
    <li>Tell them what you just told them
    </ol>
</ul>
<li><b>Dezimaltrennzeichen Komma bei csv-Dateien</b>
    <ul>
    <li>In R gibt es "read.csv" und "read.csv2"
    <li>Bei Pandas gibt es bei pd.read_csv die Parameter sep (; oder ,) und decimal (, oder .)
    </ul>
<li><b>Runden Kennwerte</b><br>
    <ul>
    <li>Beim Data Science geht es idR um das Schätzen von Parametern bei Stichproben
    <li>Die Darstellung von 16 Nachkommastellen ist eine Darstellung einer Scheingenauigkeit
    <li>Bei uns reichen idR eine Darstellung von 2-3 Ziffern (die ungleich 0 sind) aus!
    </ul>
<li><b>Warnmeldungen - vor allem Futurewarning</b><br>
    <ul>
    <li>Der Code sollte so angepasst werden, dass keine Futurewarnings angezeigt werden
    <li>Dadurch wird sichergestellt, dass der Code etwas länger mit neuen Versionen läuft       
</ul>


## Alle: pd.read_csv - Spaltentrennzeichen und Dezimaltrennzeichen

<b>Bei pd.read_csv gibt es den Parameter "decimal" zur Festlegung des Dezimaltrennzeichens.<br>
Vorteil: Weniger Code + Variable erhalten den Datentyp "float" und nicht "object" -<br>
im Vergleich zum Import mit falschem Dezimaltrennzeichen, und einem Ersetzen danach.</b><br>
https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
df = pd.read_csv("Gewichtsklasse.csv", sep=';', decimal=',')
df.info()

## Alle: Codierung - Übersicht Codealternativen

<b>Label encoding</b>

In [ ]:
col = 'Fastfood'
print('Vorher: ', df.dtypes[col], list(df[col][0:10]) )
df[col] = df[col].map({'ja': 1, 'nein': 0}) # bei 'vielleicht' würde ein nan entstehen
print('Nachhher: ', df.dtypes[col], list(df[col][0:10]))

In [ ]:
col = 'Geschlecht'
print('Vorher: ',df.dtypes[col], list(df[col][0:10]))
df[col]  = LabelEncoder().fit_transform(df[col]) # bei einem dritten Wert würde auf 0, 1 und 2 codiert werden
print('Nachher: ',df.dtypes[col], list(df[col][0:10]))

In [ ]:
col = 'Transportmittel'
print('Vorher: ',df.dtypes[col], list(df[col][0:10]))
df[col] = LabelEncoder().fit_transform(df[col])
print('Nachher: ',df.dtypes[col], list(df[col][0:10]))

In [ ]:
col = 'Alkohol'
print('Vorher: ', df.dtypes[col], list(df[col][0:10]) )
df[col] = df[col].replace({'nie': 0, 'manchmal': 1, 'häufig': 2, 'täglich':3})
print('Nachhher: ', df.dtypes[col], list(df[col][0:10]))

<b>One hot encoding, dummy encoding</b>

In [ ]:
col = 'Zwischenmahlzeit'
print('Vorher: ',df.dtypes[col], list(df[col][10:20]))
df = pd.get_dummies(df, columns=[col], drop_first=False)
cols = list(df.columns[df.columns.str.contains('Zwischen')])
print('Nachher: ')
print('\n', df[cols].info())

## Gruppe 1

<b>Bei den Histogrammen - gute Wahl der Anzahl bins. Schöne Korrelationsmatrix.</b>

<b>Tipp:<br>bei der Pandasmethode corr() kann direkt auf numerische Variable eingeschränkt werden:<br> df.corr(method='pearson', numeric_only=True)</b>

### G1: Korrelation numerischer und binäre Variable

<b>Wichtigste Punkte
<ul>
<li><b>Die Formel der Pearson-Korrelation vereinfacht sich hier, und wird als <b>Punktbiseriale Korrelation</b> bezeichnet.</b>
<li>Die Korrelation drückt aus, in welche Richtung sich der Mittelwert der numerischen Variablen von der Kodierung "0" zur Kodierung "1" ändert.
<li>Der Betrag der Korrelation wird kleiner, je unbalancierter (bei konstanten Mittelwerten pro Gruppe und bei konstanter Standardabweichung) die Datensätze bzgl. der Binärvariablen sind.
<li>Der Betrag der Korrelation wird kleiner, je größer die Standardabweichung des Datensatzes ist (bei konstanten Mittelwerten pro Gruppe, und konstanter Balanciertheit)    
</ul></b>

## G1: Korrelation zwischen zwei Binärvariablen

<ul>
<li><b>Die Formel der Pearson-Korrelation vereinfacht sich hier, und wird als <b>Phi-Koeffizient</b> bezeichnet.
</ul>

In [ ]:
df.Rauchen  = df.Rauchen.map({'ja': 1, 'nein': 0})

In [ ]:
crosstable = pd.crosstab(df.Rauchen, df.Fastfood, margins=False, margins_name="Gesamt")
crosstable

In [ ]:
phi = ( (crosstable.iloc[0, 0] * crosstable.iloc[1, 1] - crosstable.iloc[0, 1] * crosstable.iloc[1, 0]) /
        (np.sqrt(((crosstable.iloc[0, 0] + crosstable.iloc[0, 1]) * (crosstable.iloc[1, 0] + crosstable.iloc[1, 1]) *      
                  (crosstable.iloc[0, 0] + crosstable.iloc[1, 0]) * (crosstable.iloc[0, 1] + crosstable.iloc[1, 1]))))   )
print(f"Phi-Koeffizient: {phi}")

<b>Bemerkung: eure Formel Phi-Koeffizient war falsch. Ihr hattet den Zähler quadrariert, dann die Wurzel gezogen, und dadurch die Wurzel zerstört.</b>

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html
df[['Rauchen','Fastfood']].corr(method='pearson')

##  Korrelation zwischen ordinalskalierten und numerischen Variablen

<ul>
<li><b>Die Verwendung der Pearson-Korrelation ist bei ordinalen Variablen "mathematisch" nicht korrekt</b>
<li>Es ist der Rangkorrelationskoeffizient (Spearman) zu verwenden.
</ul>

In [ ]:
df[['Alkohol', 'Gewicht']].corr(method = 'pearson')

In [ ]:
df[['Alkohol', 'Gewicht']].corr(method = 'spearman')

## Gruppe 2

<ul>
<li><b>Tipp: Um True/False in 0/1 umzuwandeln reicht: df = df *1</b>
</ul>

## Gruppe 3 - kNN

<ul>
<li><b>k = 2</b><br>
    Bei scikit-learn wird bei 2 Nachbarn als Tiebraeker die Klasse des ersten Nachbarn im Trainingssatz genommen.<br>
https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.KNeighborsClassifier.html<br>
Das ist sicher nicht optimal.
<li>Beim Wertebereich von k sollte statt: <b>k_values = range(2,50)</b> besser <b>k_values = range(1,51)</b> gewählt werden.<br>
<b>Bei mir wurde k=1 wurde als optimaler k-Wert erkannt!</b>
</ul>

In [ ]:
df = pd.read_csv("Gewichtsklasse.csv", delimiter=';', decimal = ',')
label_encoder = LabelEncoder()
cols2decode = ['Geschlecht', 'Alkohol', 'Fastfood', 'Kalorienzählen', 'Rauchen', 'Familie','Zwischenmahlzeit', 'Transportmittel']
df_dec = pd.get_dummies(df, columns=cols2decode, drop_first=False) * 1

In [ ]:
df_dec['Target'] = df_dec.Gewichtsklasse.apply(lambda x: 1 if 'Adipositas' in x else 0)

In [ ]:
# split encoded dataframe into train- and test-dataframe
X = df_dec.drop(columns=['Target', 'Gewichtsklasse', 'Gewicht'])
y = df_dec['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# standardize data for kNN
scaler = StandardScaler()
X_train_z = scaler.fit_transform(X_train)
X_test_z = scaler.transform(X_test)

In [ ]:
cv_scores = []
k_values = range(1,20)
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, X_train_z, y_train, cv=10, scoring='accuracy')
    cv_scores.append(scores.mean())

# used code from Jupyter Notebook "kNN_Example" Jupyter Notebook by Alexander Adrowitzer

In [ ]:
k_acc = pd.DataFrame({'k': k_values, 'Acc': cv_scores})
k_acc[k_acc['Acc'] == max(k_acc['Acc'])]

# used code from Jupyter Notebook "kNN_Example" Jupyter Notebook by Alexander Adrowitzer

In [ ]:
# create dummy classifier
dummy_clf = DummyClassifier(strategy="most_frequent")

# train model
dummy_clf.fit(X_train, y_train)

# make prediction
y_train_pred = dummy_clf.predict(X_train)
y_test_pred = dummy_clf.predict(X_test)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=2,p=2,metric='minkowski')
knn.fit(X_train_z, y_train)
y_train_pred = knn.predict(X_train_z)
y_test_pred = knn.predict(X_test_z)

<b>Einschub ROC-Kurve (TPR über FPR)</b>

https://www.evidentlyai.com/classification-metrics/explain-roc-curve#:~:text=The%20ROC%20curve%20stands%20for,False%20Positive%20rates%20(FPR).

<b>kNN(k=2)- mögliche Fälle bei berechneten Wahrscheinlichkeit des Klassifikators</b>
<ul>
<li>Wahrscheinlichkeit(positiv) = wie sicher der Klassifikator ist, dass die Klasse positiv ist
<li>wenn beide nächste Nachbarn des Test-Datenpunkts positiv => Wahrscheinlichkeit(positiv) = 1
<li>wenn beide nächste Nachbarn des Test-Datenpunkts negativ => Wahrscheinlichkeit(positiv) = 0
<li>wenn beide nächste Nachbarn des Test-Datenpunkts pos. bzw. neg. => Wahrscheinlichkeit(positiv) = 0.5
</ul>
<b>Da bedeutet für die ROC-Kurve:</b>
<ul>
<li>bei der Abfrage <b>knn.predict_proba(X_test_z)</b> gibt es nur Werte 0, 0.5, 1
<li>Idee Threshold: Wahrscheinlichkeit(positiv) >= TH => Prediction = positiv
<li>TH ist ein Parameter, der nach unseren Wünschen festgelegt werden kann!
<li>TH = -1.0 => alle Datenpunkte werden positiv geschätzt
<li>TH =  0.2 => nur Datenpunkte mit P(p) = 0.5 oder 1 werden positiv geschätzt
<li>TH =  0.7 => nur Datenpunkte mit P(p) = 1 werden positiv geschätzt
<li>TH =  1.4 => alle Datenpunkte werden negativ geschätzt
</li>
=> im ROC gibt es nur 4 verschiedene Punkte beim kNN Classifier!

In [ ]:
# die Wahrscheinlichkeiten der ersten 10 Testdatensätze: dummy und kNN
print(dummy_clf.predict_proba(X_test_z)[0:10, 1])
print(knn.predict_proba(X_test_z)[0:10, 1])

In [ ]:
# Berechnung FRP und TPR für Dummy und kNN
fpr_test_dummy, tpr_test_dummy, _ = roc_curve(y_test, dummy_clf.predict_proba(X_test)[:,1])
fpr_test_knn, tpr_test_knn, _     = roc_curve(y_test, knn.predict_proba(X_test_z)[:,1])

print(fpr_test_dummy)
print(tpr_test_knn)

In [ ]:
# create ROC curve
# https://scikit-learn.org/dev/modules/generated/sklearn.metrics.roc_curve.html

plt.plot(fpr_knn_test, tpr_knn_test, label='kNN Classifier',marker='o')
plt.plot(fpr_dummy, tpr_dummy,marker='o')
plt.xlabel('FPR (False Positive Rate)')
plt.ylabel('TPR (True Positive Rate)')
plt.title('ROC curve (Test)')
plt.grid()
plt.legend();

In [ ]:
fpr_dummy, tpr_dummy, _ = roc_curve(y_train, dummy_clf.predict_proba(X_train)[:,1])
fpr_knn_train, tpr_knn_train, _ = roc_curve(y_train, knn.predict_proba(X_train_z)[:,1])
plt.plot(fpr_knn_train, tpr_knn_train, label='kNN Classifier',marker='o')
plt.plot(fpr_dummy, tpr_dummy, marker='o')
plt.xlabel('FPR (False Positive Rate)')
plt.ylabel('TPR (True Positive Rate)')
plt.title('ROC curve (Training)')
plt.grid()
plt.legend();

<b>Warum ist ROC Kurve hier mit kNN bei den Trainingsdaten so viel besser?</b>

## Gruppe 4

<b>alles OK</b>

<b>Ergebnis:<br>
Bei dieser einfachen Versuchsanordnung war es für die Accuracy egal, ob die ordinalen Variablen ordinal codiert oder dummy codiert wurden.</b>

# Gruppe 5

<b>Kein Feedback, da keine Abgabe</b>

# Gruppe 6

Gruppe hat Aufgabe 6 (statt 7) bearbeitet.

<ul>
<li><b>Korrelationsmatrix</b>
    <ul>
    <li>Gerade bei Korrelationsmatrizen mit negativen und positiven Werten sollte eine Korrelation 0 farblich schnell entdeckt werden (weiß).
    <li>sns.heatmap(correlation_matrix, vmin=-1, vmax=1, center=0, ....
    </ul>
<li><b>Kopieren DataFrame</b>
    <ul>
    <li>df_encoded = df.copy()
    <li><b>Mögliche sinnvolle Strategie:</b><br>
        df sollte möglichst nah bei den Rohdaten bleiben, da man dann immer darauf zurückgreifen kann, ohne neu zu laden<br>
        Bei verschiedenen Änderungen an df jeweils eine neue (sprechende) Bezeichnung verwenden
</ul>

In [ ]:
# Dieser Code ist hier nicht lauffähig!!!
'''
binaer_yes_no_features = ['Fastfood', 'Kalorienzählen', 'Rauchen', 'Familie']

# Diese Schleife ist nicht nötig:
for e in binaer_yes_no_features:
    df_encoded[e] = df_encoded[e].replace({'ja': 1, 'nein': 0})

# Das geht so ohne Schleife:
df_encoded[binaer_yes_no_feature] = df_encoded[binaer_yes_no_feature].replace({'ja': 1, 'nein': 0})
'''

<b>Zusatzbemerkung zu randomizedsearchcv() (für alle!)<br>
Es ist möglich, Hyperparameter zufällig aus Wahrscheinlichkeitsverteilungen zu ziehen (bei Gridsearch nicht möglich)</b>

In [ ]:
import random as rd
param_dist = {
   'C':      rd.uniform(0.1, 10),  # Uniform distribution between 0.1 and 10
   'kernel': ['linear', 'rbf', 'poly'],
   'gamma':  ['scale', 'auto'] + list(np.logspace(-3, 3, 50))
}

<b>Feature Importances beim Random Forest</b><br>
https://scikit-learn.org/1.5/auto_examples/ensemble/plot_forest_importances.html

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

y_pred_train = rf.predict(X_train)
y_pred_test  = rf.predict(X_test)

acc_test = accuracy_score(y_test, y_pred_test)
print('Accuracy(test)= ',round(acc_test*100,1),'%')

In [ ]:
feature_names = X.columns
importances = rf.feature_importances_
forest_importances = pd.Series(importances, index=feature_names)
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)   # Standardabweichung der Importances pro Baum

In [ ]:
fig, ax = plt.subplots()
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title('Feature importances using mean decrease impurity (MDI)')
ax.set_ylabel('MDI')
fig.tight_layout()

# Aufgabe 7 PCA - wurde nicht vorgestellt

https://scikit-learn.org/dev/modules/generated/sklearn.decomposition.PCA.html

<b>Standardsierung - was macht die Methode PCA() ?</b><br>

<b>Linear dimensionality reduction using Singular Value Decomposition of the data to project it to a lower dimensional space.<br>
The input data is <u>centered but not scaled</u> for each feature before applying the SVD.</b>

<b>Standardsierung - was sollte man machen?</b><br>
https://medium.com/@roshmitadey/understanding-principal-component-analysis-pca-d4bb40e12d33#:~:text=Before%20performing%20PCA%2C%20it's%20essential,equal%20importance%20in%20the%20analysis.
Before performing PCA, it’s essential to standardize the data.<br>
This means <b>centering the data by subtracting the mean and scaling it by dividing by the standard deviation</b>.<br>
Standardization ensures that all features have equal importance in the analysis.

In [ ]:
X_train_stand = scaler.fit_transform(X_train)
X_test_stand  = scaler.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
X_train_stand.shape

In [ ]:
# Wir haben 31 features => maximal 30 Hauptkomonenten sind erzeugbar!
pca         = PCA(n_components = X_train_stand.shape[1])
X_train_pca = pca.fit_transform(X_train_stand)
X_test_pca  = pca.transform(X_test_stand)

<b>Eigenschaften der Hauptkomponenten</b>

In [ ]:
# diese sind mittelwertfrei
print(X_train_pca[:,0].mean())
print(X_train_pca[:,1].mean())
print(X_train_pca[:,2].mean())

In [ ]:
# die Varianz der einzelnen Hauptkomponenten nimmt ab
print(round(np.var(X_train_pca[:,0]),2))
print(round(np.var(X_train_pca[:,1]),2))
print(round(np.var(X_train_pca[:,2]),2))

In [ ]:
# die einzelnen Hauptkomponenten stehen senkrecht aufeinander (Skalarprodukt der Vektoren ist null)
print(np.dot(X_train_pca[:,0],X_train_pca[:,1]))
print(np.dot(X_train_pca[:,0],X_train_pca[:,2]))
print(np.dot(X_train_pca[:,1],X_train_pca[:,2]))

In [ ]:
# der Korrelationskoeffizent zwischen den Hauptkomponenten ist dann natürlich auch 0
print(np.corrcoef(X_train_pca[:,0],X_train_pca[:,1])[0,1])
print(np.corrcoef(X_train_pca[:,0],X_train_pca[:,2])[0,1])

In [ ]:
plt.figure(figsize=(6,5))
plt.scatter(X_train_pca[:,0], X_train_pca[:,1], c= y_train, cmap='plasma')
plt.xlabel('Erste Hauptkomponente')
plt.ylabel('Zweite Hauptkomponente')
plt.title('Klassenverteilung der ersten beiden Hauptkomponenten')
plt.legend(['adipös','nicht adipös']);

<b>SVC + PCA</b>

In [ ]:
from sklearn import svm

In [ ]:
# SVC ohne PCA zum vergleichen
svc = svm.SVC() # alles default
svc.fit(X_train, y_train)
y_pred_test  = svc.predict(X_test)
acc_test     = accuracy_score(y_test, y_pred_test)
print('Accuracy(test)= ',round(acc_test*100,1),'%')

In [ ]:
# Sammlung Accuracy für 1 bis 31 Hauptkomponenten
acc_test = list()

for i in range(1, X_train_stand.shape[1]+1):
    svc.fit(X_train_pca[:,0:i], y_train)
    y_pred_test  = svc.predict(X_test_pca[:,0:i])
    y_pred_train = svc.predict(X_train_pca[:,0:i])
    acc_test.append(accuracy_score(y_test, y_pred_test))

In [ ]:
plt.plot(range(1, X_train_stand.shape[1]+1),acc_test, marker='o')
plt.title('Accuracy über Anzahl Hauptkomponenten (SVC)')
plt.grid();

In [ ]:
# SVC ohne PCA
svc.fit(X_train, y_train)
y_pred_test  = svc.predict(X_test)
print('Confusionmatrix(Test)\n ',confusion_matrix(y_test, y_pred_test))
print('Accuray(Test) ',round(accuracy_score(y_test, y_pred_test),3))
round(acc_test,3)

In [ ]:
# 5 Hauptkomponenten
svc.fit(X_train_pca[:,0:5], y_train)
y_pred_test  = svc.predict(X_test_pca[:,0:5])
print('Confusionmatrix(Test)\n ',confusion_matrix(y_test, y_pred_test))
print('Accuray(Test) ',round(accuracy_score(y_test, y_pred_test),3))

In [ ]:
# 22 Hauptkomponenten
svc.fit(X_train_pca[:,0:22], y_train)
y_pred_test  = svc.predict(X_test_pca[:,0:22])
print('Confusionmatrix(Test)\n ',confusion_matrix(y_test, y_pred_test))
print('Accuray(Test) ',round(accuracy_score(y_test, y_pred_test),3))

# Gruppe 7 (Voting, Bagging, Pasting)

<b>Alles OK</b>

# Übersicht Ergebnisse Binäre Klassifikation

\begin{array}{|c|c|} \hline
\text{Klassifizierer} & \text{Accuracy} \\ \hline \hline
\text{Dummy} & 55\% \\ \hline
\text{SVC} & 76\%  \\ \hline
\text{Log. Regression} & 79\%  \\ \hline
\text{Decision tree} & 88\%  \\ \hline
\text{SVC+PCA} & 89\%  \\ \hline
\text{kNN (k=5)} & 92\%  \\ \hline
\text{Decision tree (pasting)} & 92\%  \\ \hline
\text{Decision tree (bagging)} & 94\%  \\ \hline
\text{Random Forest} & 94\%  \\ \hline
\end{array}

<b>Für vertrauenswürdigere Ergebnisse sind Hyperparameteroptimierungen, Feature Selection und mehrfache Train-Test-Splits vorzunehmen.</b>

# Gruppe 8 (Mehrklassen-Klassifizierung)

<ul>
<li><b>Korrelationsmatrix</b>
    <ul>
    <li>Gerade bei Korrelationsmatrizen mit negativen und positiven Werten sollte eine Korrelation 0 farblich schnell entdeckt werden (weiß).
    <li>sns.heatmap(correlation_matrix, vmin=-1, vmax=1, center=0, ....
    </li>
    </ul>
<li><b>ROC im Multiklassen-Fall</b>
    <ul>
    <li>Auch in diesem Fall ist es möglich, ROCs zu erzeugen, siehe<br>
        https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
    </ul>

# Gruppe 9

<ul>
<li><b>Grafiken</b>
    <ul>
    <li>Bei den Histogrammen sind die Anzahl Bins zu klein
    <li>Beschriftungen der x-Achse überlappen sich
    <li>Gut: Verwendung subplots
    </ul>
<li><b>Codierung</b>
    <ul>
    <li>Schleifen über Spalten sind im Pandas nicht nötig
</ul>

# Gruppe 10

<ul>
<li><b>Codierung der Gewichtsklasse ist unnötig</b>
<li><b>Schleifen über Spalten</b>
    <ul>
    <li>Schleifen über Spalten sind im Pandas nicht nötig
    </li>
    </ul>
<li><b>Verwendung Spalten</b>
    <ul>
    <li>Spalten 'Alkohol', 'Zwischenmahlzeit', 'Transportmittel' wurden ohne Begründung gelöscht.<br>
        Löschen von Daten ist immer zu begründen.
    </ul>
<li><b>Codeverweise</b>
    <ul>
    <li>Ich denke, die Funktionen habt ihr nicht selbst erstellt, dh Verweis fehlt!
    <li>Aber schöne Grafiken
    </ul>
</ul>

<b>Wie ihr das im Folgenden gemacht habt, sollte das NICHT gemacht werden:</b>

In [ ]:
'''
sklearn_lda = LDA()                                    
X_lda_sklearn = sklearn_lda.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_lda_sklearn, y, random_state=147, test_size = 0.2, shuffle=True)
'''

<b>Auf diese Weise sollte das geschehen:</b>

In [ ]:
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc = StandardScaler()
X_train_z = sc.fit_transform(X_train)
X_test_z  = sc.transform(X_test)

lda = LDA()
X_train_lda = lda.fit_transform(X_train_z, y_train_z)
X_test_lda  = lda.transform(X_test_z)
'''

<b>range(1,6) erzeugt Integer von 1-5 - daher fehlt 6. Komponente.</b>

In [ ]:
for n in range(1, 6):
    print(n)